## Selenium 활용 데이터 수집 - 파파고

## 마크다운파일(md) 크롬에서 보는 방법 
구글 웹스토어에서 markdown viewer 설치 -> m 버튼 눌러주고 (구글창 오른쪽 사이드 위) -> 설정 -> 확장프로그램 -> markdown viewer -> 세부정보 -> 파일 URL에 대한 액세스 허용 켜기, 사이트 엑세스 켜기 -> 마크다운 연결프로그램 다른앱선택 (구글>크롬)

In [22]:
import chromedriver_autoinstaller #ChromeDriver 자동 설치 모듈
from selenium import webdriver # Chrome을 제어하기 위한 객체
from selenium.webdriver.support.ui import WebDriverWait # Chrome이 웹페이지 로딩을 완료할 때까지 최대 n초간 대기하는 기능
from bs4 import BeautifulSoup
from pandas import DataFrame 
import time # 지정된 시간동안 파이썬에 렉을 거는 기능 
from selenium.webdriver.common.by import By # 원하는 부분을 추출할 수 있는 방식을 지정하는 객체
import time
import datetime as dt

### 번역할 파일 가져오기

번역할 문장이 행 단위로 입력되어 있는 영문 텍스트 파일이 필요함

In [23]:
source = None
with open('news.txt','r',encoding='utf-8') as f :
    source = f.readlines()


['Desperately projecting that everything is as it was, the Kremlin is only emphasizing how much has changed.\n',
 '\n',
 'These were 36 hours that provided a glimpse of the end of Russian President Vladimir Putin’s rule. Almost every action was improbable, at best a week ago – much was inconceivable, 17 months ago.\n',
 '\n',
 'A loyal henchman slams the premise of the invasion, then claims an airstrike targeted his troops, before taking a major military town without clashes, Prigozhin said, and then marching to within a few hundred miles of Moscow. But suddenly he executes a baffling reversal, turning back to avoid bloodshed, as the Kremlin claims Alexander Lukashenko, the president of Belarus whom Putin seems to treat with contempt, brokered a dramatic reprieve, in which the insurrectionist who has his armor bound for Moscow, now opts for exile in Minsk.\n',
 '\n',
 'Even as the dust settles, it still makes little sense. It is important to remember we have yet to hear from Yevgeny Pr

웹브라우저 준비

In [24]:
# 크롬 드라이버 자동 설치
chromedriver_autoinstaller.install()

# 크롬드라이버를 통해 크롬을 실행시킴
driver = webdriver.Chrome() # driverㄴ 객체는 Chrome 자체 

# 크롬 브라우저가 준비될 때까지 최대 5초씩 대기해라 
driver.implicitly_wait(5)

# 네이버 로그인 페이지로 이동
driver.get('https://papago.naver.com/')

# 페이지 로딩이 완료될 때까지 1초간 대기 
time.sleep(3)


번역 수행하기

In [28]:
# 반복 수행 중 이번 회차에서 번역한 결과를
currentTrans = None
# 이번에 반복한 결과를
prevTranse = None

# 번역 결과가 저장될 텍스트 파일을 분초단위로 생성한다.
filename = dt.datetime.now().strftime("%y%m%d_%H%M%S.md")



'''
흐름 
텍스트를 입력하는 영역을 찍어보면 txtSource 가 id
파파고의 기존 인용 내용을 항상 지우고 시작해야 한다.
그리고 한 줄씩 입력
그리고 반역하기 버튼의 객체를 가져와서 클릭
번역 돌아갈 시간 주고 
번역내용의 id는 TxtTarget
번역 복사할 시간 주고
번역 결과가 저장될 텍스트 파일을 생성하고 
저장한다.

'''
with open(filename,'w',encoding='utf-8') as f :
    # 읽어온 텍스트의 원소만큼 반복한다.
    for s in source : # s는 원문
        # 각 행마다 줄바꿈 문자 등의 앞뒤 공백을 제거한다.
        s = s.strip()

        # 이번 턴에서 번역할 내용이 빈 행이라면 넘어감 
        if not s :
            continue

        #print(s) # 줄 글로(영어) 쭉 읽어옴 

        # 번역할 원문을 입력하는 요소를 취득한다.
        txtSource = WebDriverWait(driver,3).until(lambda x : x.find_element(By.CSS_SELECTOR,"#txtSource"))

        # 기존의 입력 내용을 삭제 한다. (그래야 새로운 영어를 붙여넣기 하니까)
        txtSource.clear()

        # 번역할 한 행을 입력요소에 키보드로 입력한다.
        txtSource.send_keys(s)

        # 번역하기 버튼 요소 취득
        btnTranslate = WebDriverWait(driver,3).until(lambda x : x.find_element(By.CSS_SELECTOR,"#btnTranslate"))

        # 번역 버튼 객체를 가져와서 클릭한다.
        btnTranslate.click()

        # 번역이 되는 1초가나 대기한다.
        time.sleep(1)
# ---------------------------------------------------------------------------------
        while True :
            # 번역 결과가 표시될 요소를 취득
            txtTarget = WebDriverWait(driver,3).until(lambda x : x.find_element(By.CSS_SELECTOR,"#txtTarget"))
            
            # 번역 결과 가져오기
            currentTrans = txtTarget.text.strip()
            #print(currentTrans)

            # 가져온 내용이 있고 이번 번역결과가 이전 번역결과와 다르다면 반복해라
            if currentTrans and currentTrans != prevTranse :
                prevTranse = currentTrans 
                break
            
            time.sleep(1)
# ---------------------------------------------------------------------------------
        f.write(s)
        f.write('\n\n')
        f.write("> ") # 인용구 
        f.write(currentTrans)
        f.write('\n\n')
'''
업그레이드 버젼 
이번 회차에서 번역한 결과 C와 이전 번역 결과P
기본값은 둘다 None 
결과가 다를 때 break하고 반복문을 탈출한다.
이전 번역 결과를 현재에 계속 담는다. 랙 걸리는걸 회피하기 위해서 이렇게함.
'''    
        
